In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools1 import MCSimulation

In [3]:
load_dotenv()

True

In [4]:
api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")

In [5]:
alpaca = tradeapi.REST(
    api_key,
    secret_key,
    api_version="v2")

In [10]:
start_date = pd.Timestamp('2018-06-15', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2023-06-15', tz='America/New_York').isoformat()

tickers = ["SHOP", "SSY", "FDX", "RIO", "HD", "SBUX", "SOFI", "SPY"]

timeframe = "1Day"

df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

df_portfolio

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2018-06-15 04:00:00+00:00,262.5100,264.9900,260.8434,264.5600,1363654,15540,263.539563,FDX
2018-06-18 04:00:00+00:00,263.0900,264.6500,261.5800,263.6700,1399778,16116,263.381021,FDX
2018-06-19 04:00:00+00:00,261.4300,261.5100,254.7500,258.3900,3160686,33506,258.404889,FDX
2018-06-20 04:00:00+00:00,257.3100,257.3100,251.0900,251.4300,3906190,45397,253.012027,FDX
2018-06-21 04:00:00+00:00,251.5000,251.8000,245.6480,246.7200,3671626,32548,247.837172,FDX
...,...,...,...,...,...,...,...,...
2023-06-09 04:00:00+00:00,0.9723,0.9850,0.9300,0.9310,8670,60,0.968447,SSY
2023-06-12 04:00:00+00:00,0.9310,0.9900,0.8800,0.8802,36532,111,0.920997,SSY
2023-06-13 04:00:00+00:00,0.8800,0.8800,0.8512,0.8749,33655,145,0.864170,SSY


In [14]:
SHOP = df_portfolio[df_portfolio['symbol']=='SHOP'].drop(['symbol', 'trade_count', 'vwap', 'high', 'low', 'volume', 'open'], axis=1)
SSY = df_portfolio[df_portfolio['symbol']=='SSY'].drop(['symbol', 'trade_count', 'vwap', 'high', 'low', 'volume', 'open'], axis=1)
FDX = df_portfolio[df_portfolio['symbol']=='FDX'].drop(['symbol', 'trade_count', 'vwap', 'high', 'low', 'volume', 'open'], axis=1)
RIO = df_portfolio[df_portfolio['symbol']=='RIO'].drop(['symbol', 'trade_count', 'vwap', 'high', 'low', 'volume', 'open'], axis=1)
HD = df_portfolio[df_portfolio['symbol']=='HD'].drop(['symbol', 'trade_count', 'vwap', 'high', 'low', 'volume', 'open'], axis=1)
SBUX = df_portfolio[df_portfolio['symbol']=='SBUX'].drop(['symbol', 'trade_count', 'vwap', 'high', 'low', 'volume', 'open'], axis=1)
SOFI = df_portfolio[df_portfolio['symbol']=='SOFI'].drop(['symbol', 'trade_count', 'vwap', 'high', 'low', 'volume', 'open'], axis=1)

SPY_df = df_portfolio[df_portfolio['symbol']=='SPY'].drop(['symbol', 'trade_count', 'vwap', 'high', 'low', 'volume', 'open'], axis=1)

stock_data_df = pd.concat([SHOP, SSY, FDX, RIO, HD, SBUX, SOFI], 
            axis=1, keys=["SHOP", "SSY", "FDX", "RIO", "HD", "SBUX", "SOFI"])

stock_data_df.index = stock_data_df.index.date

display(stock_data_df)
display(SPY_df)


,SHOP,SSY,FDX,RIO,HD,SBUX,SOFI
,close,close,close,close,close,close,close
2018-06-15,168.44,1.2800,264.56,56.79,200.54,57.11,NaN
2018-06-18,172.84,1.3900,263.67,57.06,200.69,57.22,NaN
2018-06-19,170.61,1.3200,258.39,54.95,199.21,57.43,NaN
2018-06-20,172.25,1.3800,251.43,55.07,199.08,52.22,NaN
2018-06-21,164.54,1.3200,246.72,54.72,200.31,50.62,NaN
...,...,...,...,...,...,...,...
2023-06-09,61.36,0.9310,223.76,64.57,297.35,97.94,8.18
2023-06-12,65.01,0.8802,224.55,63.74,300.86,98.48,9.16
2023-06-13,63.84,0.8749,228.87,65.60,300.08,99.26,9.54


,close
timestamp,
2018-06-15 04:00:00+00:00,277.03
2018-06-18 04:00:00+00:00,276.59
2018-06-19 04:00:00+00:00,275.48
2018-06-20 04:00:00+00:00,276.01
2018-06-21 04:00:00+00:00,274.24
...,...
2023-06-09 04:00:00+00:00,429.90
2023-06-12 04:00:00+00:00,433.80
2023-06-13 04:00:00+00:00,436.66
